##  LX 판토스 공장 시뮬레이션용 냉동창고 더미데이터 생성 코드 설명

본 코드는 **냉동/냉장창고 시뮬레이션**을 위한 더미 센서 데이터를 생성하고, 구역(A, B, C, D)별로 Excel 파일로 저장하는 과정입니다. 시계열 기반의 데이터로, 실제 공장의 환경을 가정하여 센서 반응을 반영했습니다.

---

## 공장설명
| 구역         | 용도                    | 면적 (예시) |
| ---------- | --------------------- | ------- |
| **Zone A** | 입고·출고 지역 (도어 접점 다수)   | 100 ㎡   |
| **Zone B** | 냉장 보관 구역 (0\~-5℃ 유지)  | 200 ㎡   |
| **Zone C** | 냉동 보관 구역 (-18℃ 이하 유지) | 300 ㎡   |
| **Zone D** | 장기 보관 또는 예비 구역        | 150 ㎡   |



---
###  주요 기능 요약

- **구역(zone)**: A-1 ~ D-3 총 12개 구역
- **시간 범위**: 2025년 1월 1일부터 **30일치 1시간 간격 시계열 데이터** (총 720시간)
- **센서 항목**
  - `temperature`: 온도 (℃)
  - `humidity`: 습도 (%)
  - `co2`: 이산화탄소 농도 (ppm)
  - `door_open`: 문 열림 여부 (0/1)
  - `stock_level`: 재고 비율 (0~1)
  - `cooler_output_ratio`: 냉각기 출력 비율 (0~1)

---

###  센서 동작 로직

- **온도 변화**  
  - 구역별 기본 평균 온도를 기준으로 생성
  - `door_open == 1`일 경우 **1.5~2.5℃ 상승** 반영

- **CO₂ 농도**  
  - `stock_level`이 높을수록 **50ppm까지 가산**
  - 약간의 무작위 노이즈 포함

- **냉각기 출력률**  
  - 온도가 높거나 문이 자주 열릴수록 **출력 비율 증가**
  - 출력 비율은 0~1 사이로 제한됨

In [7]:
import pandas as pd
import numpy as np
import os

save_dir = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시"
os.makedirs(save_dir, exist_ok=True)  # 폴더 없으면 생성

# 시간 생성 (1시간 간격, 7일치)
date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')
zones = [f"{zone}-{i}" for zone in ['A', 'B', 'C', 'D'] for i in range(1, 4)]

# 구역별 평균 온도
base_temp_mean = {'A': 10, 'B': 2, 'C': -18, 'D': 5}

# 센서 데이터 생성 함수
def generate_sensor_data(zone, n_hours):
    zone_prefix = zone.split('-')[0]
    zone_index = int(zone.split('-')[1])
    rng = np.random.default_rng(seed=zone_index + ord(zone_prefix))  # 확실히 다른 seed

    # 기본값 설정
    temp_mean = base_temp_mean[zone_prefix]
    temp = rng.normal(loc=temp_mean, scale=2, size=n_hours)

    # 문 열림 여부
    door_open = rng.binomial(1, 0.2 if zone_prefix == 'A' else 0.05, n_hours)

    # 문 열림 시 온도 상승 (약간)
    temp += door_open * rng.uniform(1.5, 2.5, n_hours)  # 1.5~2.5도 상승

    # 습도
    humidity = rng.uniform(50, 85, n_hours)

    # 재고 수준
    stock = np.clip(np.cumsum(rng.normal(0, 0.01, n_hours)) + 0.5, 0, 1)

    # CO2 농도 (재고 높을수록 약간 상승 경향)
    co2_base = 400 + stock * 50
    co2 = co2_base + rng.normal(0, 25, n_hours)

    # 냉각기 출력률 = 온도↑ 문열림↑ → 출력 비율↑
    cooler_output_ratio = np.clip(
        1 - (temp_mean - temp) / 15 + 0.1 * door_open + rng.normal(0, 0.03, n_hours),
        0, 1
    )

    return pd.DataFrame({
        'zone': zone,
        'datetime': date_range,
        'temperature': temp,
        'humidity': humidity,
        'co2': co2,
        'door_open': door_open,
        'stock_level': stock,
        'cooler_output_ratio': cooler_output_ratio
    })

# 구역별로 나눠서 저장
for group in ['A', 'B', 'C', 'D']:
    sub_zones = [z for z in zones if z.startswith(group)]
    df_group = pd.concat([generate_sensor_data(z, len(date_range)) for z in sub_zones], ignore_index=True)

    file_name = f"냉동창고_더미데이터_시계열_{group}.xlsx"
    file_path = os.path.join(save_dir, file_name)
    df_group.to_excel(file_path, index=False)
    print(f"{file_path} 저장 완료")

C:\Users\82108\AppData\Local\Temp\ipykernel_29488\2236217129.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')


C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_더미데이터_시계열_A.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_더미데이터_시계열_B.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_더미데이터_시계열_C.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_더미데이터_시계열_D.xlsx 저장 완료


```python
# 🌤 외부 기후 데이터 생성
# 4주 주기로 각 계절(봄→여름→가을→겨울)을 반복합니다.
weeks = len(date_range) // (24*7)
spring = np.linspace(5, 15, 24*7)     # 봄: 5~15℃
summer = np.linspace(22, 30, 24*7)    # 여름: 22~30℃
autumn = np.linspace(10, 18, 24*7)    # 가을: 10~18℃
winter = np.linspace(-5, 5, 24*7)     # 겨울: -5~5℃

seq = np.concatenate([spring, summer, autumn, winter])
outdoor_temp = np.tile(seq, weeks + 1)[:len(date_range)]
outdoor_hum  = np.tile(np.linspace(30, 90, 24*7), weeks + 1)[:len(date_range)]
outdoor_co2  = rng.normal(400, 10, len(date_range))  # 외부 CO₂ 농도

# 🏭 내부 초기값 생성
temp        = rng.normal(base_temp_mean[prefix], 2, size=len(date_range))
humidity    = rng.uniform(50, 85, size=len(date_range))
stock       = np.clip(np.cumsum(rng.normal(0,0.01,len(date_range))) + 0.5, 0, 1)
co2         = 400 + stock * 50 + rng.normal(0,25,len(date_range))
door_open   = rng.binomial(1, 0.1, size=len(date_range))
open_dur    = door_open * rng.uniform(5, 60, size=len(date_range))
power_eff   = rng.uniform(0.7, 1.2)

cooler_out = np.zeros(len(date_range))

# 🔄 시간 순환 보정 루프
for i in range(len(date_range)):
    # 1) 문 열림 시 내부 온도 즉각 상승 (1.5~2.5℃ 랜덤)
    if door_open[i]:
        temp[i] += rng.uniform(1.5, 2.5)

    # 2) 외부 온도와 단열 수준 고려한 보정
    ins_factor = {'상':0.2, '중':0.5, '하':1.0}[insulation_levels[prefix]]
    temp_diff = outdoor_temp[i] - temp[i]
    temp[i] += temp_diff * ins_factor * 0.05

    # 3) 냉각기 출력률 계산
    cooler_out[i] = np.clip(
        (temp[i] - base_temp_mean[prefix]) * 0.1 +
        temp_diff * 0.02 +
        door_open[i] * 0.05,
        0, 1
    ) / power_eff

    # 냉각 효과로 내부 온도 즉시 감소
    temp[i] -= cooler_out[i] * 1.0

    # 4) 습도 조정: 외부 습도, 문 열림, 냉각 제습 효과 고려
    humidity[i] += (outdoor_hum[i] - humidity[i]) * 0.05 + door_open[i] * 1.0
    humidity[i] -= cooler_out[i] * 5

    # 5) CO₂ 조정: 재고량 증가 vs 환기 효과, 외부 기준 유지
    co2[i] += stock[i] * 20 - door_open[i] * 30
    co2[i] = max(co2[i], outdoor_co2[i])


In [11]:
import pandas as pd
import numpy as np
import os

save_dir = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시"
os.makedirs(save_dir, exist_ok=True)

date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')
zones = [f"{z}-{i}" for z in ['A','B','C','D'] for i in range(1,4)]

base_temp_mean = {'A':10, 'B':2, 'C':-18, 'D':5}
storage_sizes = {'1':'소형', '2':'중형', '3':'대형'}
insulation_levels = {'A':'상', 'B':'중', 'C':'하', 'D':'중'}
storage_power_factor = {'소형':1.0, '중형':1.5, '대형':2.0}

def generate_zone_data(zone):
    prefix, idx = zone.split('-')
    rng = np.random.default_rng(seed=int(idx) + ord(prefix))

    weeks = len(date_range) // (24*7)
    seq_temp = np.concatenate([
        np.linspace(5, 15, 24*7),
        np.linspace(22, 30, 24*7),
        np.linspace(10, 18, 24*7),
        np.linspace(-5, 5, 24*7)
    ])
    outdoor_temp = np.tile(seq_temp, weeks+1)[:len(date_range)]

    seq_hum = np.concatenate([
        np.linspace(40, 60, 24*7),
        np.linspace(70, 90, 24*7),
        np.linspace(40, 60, 24*7),
        np.linspace(30, 50, 24*7)
    ])
    outdoor_hum = np.tile(seq_hum, weeks+1)[:len(date_range)]
    outdoor_co2 = rng.normal(400, 10, len(date_range))

    temp = np.full(len(date_range), base_temp_mean[prefix], dtype=float)
    humidity = np.full(len(date_range), 60.0, dtype=float)
    stock = np.clip(np.cumsum(rng.normal(0,0.01,len(date_range)))+0.5, 0,1)
    co2 = 400 + stock*50 + rng.normal(0,25,len(date_range))
    door_open = rng.binomial(1, 0.5, len(date_range))
    open_duration_minutes = door_open * rng.uniform(5,60,len(date_range))
    power_efficiency = rng.uniform(0.7, 1.2)

    cooler_output_ratio = np.zeros(len(date_range))
    heater_output_ratio = np.zeros(len(date_range))
    cooler_power_kWh = np.zeros(len(date_range))
    heater_power_kWh = np.zeros(len(date_range))
    total_power_kWh = np.zeros(len(date_range))
    carbon_emission_kg = np.zeros(len(date_range))

    size = storage_sizes[idx]
    power_factor = storage_power_factor[size]
    ins_factor = {'상': 0.2, '중': 0.5, '하': 1.0}[insulation_levels[prefix]]

    for i in range(len(date_range)):
        prev_temp = temp[i-1] if i > 0 else base_temp_mean[prefix]
        prev_hum = humidity[i-1] if i > 0 else 60.0
        duration_hours = open_duration_minutes[i] / 60

        if door_open[i]:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.6 * duration_hours
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.5 * duration_hours
        else:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.15
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.05

        temp_diff = temp[i] - base_temp_mean[prefix]

        if temp_diff > 0.5:
            cooler_out = ((temp_diff * 0.4) + (door_open[i] * 0.4) + (duration_hours * 0.3))
            cooler_out = np.clip(cooler_out, 0.0, 1.0) / power_efficiency
            temp[i] -= cooler_out * 5.0
            cooler_output_ratio[i] = cooler_out
            cooler_power_kWh[i] = cooler_out * power_factor * 1.5
        elif temp_diff < -0.5:
            heater_out = ((-temp_diff * 0.4) + (door_open[i] * 0.4) + (duration_hours * 0.3))
            heater_out = np.clip(heater_out, 0.0, 1.0) / power_efficiency
            temp[i] += heater_out * 2.0
            heater_output_ratio[i] = heater_out
            heater_power_kWh[i] = heater_out * power_factor

        total_power_kWh[i] = cooler_power_kWh[i] + heater_power_kWh[i]
        carbon_emission_kg[i] = total_power_kWh[i] * 0.5
        humidity[i] = np.clip(humidity[i] - cooler_output_ratio[i]*4.5, 20, 100)
        co2[i] += stock[i]*20 - door_open[i]*30
        co2[i] = max(co2[i], outdoor_co2[i])

    return pd.DataFrame({
        'datetime': date_range,
        'zone': zone,
        'temperature': temp,
        'humidity': humidity,
        'co2': co2,
        'door_open': door_open,
        'open_duration_minutes': open_duration_minutes,
        'stock_level': stock,
        'cooler_output_ratio': cooler_output_ratio,
        'heater_output_ratio': heater_output_ratio,
        'cooler_power_kWh': cooler_power_kWh,
        'heater_power_kWh': heater_power_kWh,
        'total_power_kWh': total_power_kWh,
        'carbon_emission_kg': carbon_emission_kg,
        'outdoor_temperature': outdoor_temp,
        'outdoor_humidity': outdoor_hum,
        'outdoor_co2': outdoor_co2,
        'storage_size': size,
        'insulation_grade': insulation_levels[prefix],
        'power_efficiency': power_efficiency
    })

# 저장 + 색상 강조
for prefix in ['A', 'B', 'C', 'D']:
    df_list = [generate_zone_data(f"{prefix}-{i}") for i in range(1, 4)]
    df_zone = pd.concat(df_list, ignore_index=True)
    file_path = os.path.join(save_dir, f"냉동창고_시뮬_{prefix}.xlsx")

    def color_cells(row):
        return [
            'background-color: yellow' if col in ['temperature', 'outdoor_temperature']
            else 'background-color: lightblue' if col in ['cooler_output_ratio', 'heater_output_ratio']
            else '' for col in row.index
        ]

    styled = df_zone.style.apply(color_cells, axis=1)
    styled.to_excel(file_path, index=False, engine='openpyxl')
    print(f"{file_path} 저장 완료")


C:\Users\82108\AppData\Local\Temp\ipykernel_17600\4069121425.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')


C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_A.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_B.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_C.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_D.xlsx 저장 완료


## 수정본

In [20]:
import pandas as pd
import numpy as np
import os

# 저장 경로 설정
save_dir = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시"
os.makedirs(save_dir, exist_ok=True)

# 시뮬레이션 기간: 30일 (720시간), 시간 간격: 1시간
date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')

# 구역 이름 생성: A-1 ~ D-3
zones = [f"{z}-{i}" for z in ['A','B','C','D'] for i in range(1,4)]

# 각 구역에 대한 기본 설정값
base_temp_mean = {'A':10, 'B':2, 'C':-18, 'D':5}
storage_sizes = {'1':'소형', '2':'중형', '3':'대형'}
insulation_levels = {'A':'상', 'B':'중', 'C':'하', 'D':'중'}
storage_power_factor = {'소형':1.0, '중형':1.5, '대형':2.0}

def generate_zone_data(zone):
    prefix, idx = zone.split('-')
    rng = np.random.default_rng(seed=int(idx) + ord(prefix))

    # 외부 온도/습도/CO2 시계열 생성 (4계절 반복)
    weeks = len(date_range) // (24*7)
    seq_temp = np.concatenate([
        np.linspace(5, 15, 24*7),     # 봄
        np.linspace(22, 30, 24*7),    # 여름
        np.linspace(10, 18, 24*7),    # 가을
        np.linspace(-5, 5, 24*7)      # 겨울
    ])
    outdoor_temp = np.tile(seq_temp, weeks+1)[:len(date_range)]

    seq_hum = np.concatenate([
        np.linspace(40, 60, 24*7),
        np.linspace(70, 90, 24*7),
        np.linspace(40, 60, 24*7),
        np.linspace(30, 50, 24*7)
    ])
    outdoor_hum = np.tile(seq_hum, weeks+1)[:len(date_range)]
    outdoor_co2 = rng.normal(400, 10, len(date_range))

    # 내부 상태 변수 초기화
    temp = np.full(len(date_range), base_temp_mean[prefix], dtype=float)
    humidity = np.full(len(date_range), 60.0, dtype=float)
    stock = np.clip(np.cumsum(rng.normal(0, 0.01, len(date_range))) + 0.5, 0, 1)
    co2 = 400 + stock*50 + rng.normal(0, 25, len(date_range))
    door_open = rng.binomial(1, 0.5, len(date_range))
    open_duration_minutes = door_open * rng.uniform(5, 60, len(date_range))
    power_efficiency = rng.uniform(0.7, 1.2)

    # 출력 변수 초기화
    cooler_output_ratio = np.zeros(len(date_range))
    heater_output_ratio = np.zeros(len(date_range))
    cooler_power_kWh = np.zeros(len(date_range))
    heater_power_kWh = np.zeros(len(date_range))
    total_power_kWh = np.zeros(len(date_range))
    carbon_emission_kg = np.zeros(len(date_range))

    # 설정값 가져오기
    size = storage_sizes[idx]
    power_factor = storage_power_factor[size]
    ins_factor = {'상': 0.2, '중': 0.5, '하': 1.0}[insulation_levels[prefix]]

    # 시간 순회하면서 시뮬레이션
    for i in range(len(date_range)):
        prev_temp = temp[i-1] if i > 0 else base_temp_mean[prefix]
        prev_hum = humidity[i-1] if i > 0 else 60.0
        duration_hours = open_duration_minutes[i] / 60

        # 기본적으로 외부와의 단열에 따라 내부 온도는 자연스럽게 상승
        if door_open[i]:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.6 * duration_hours
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.5 * duration_hours
        else:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.15  # 자연 상승 계수 높임
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.05

        temp_diff = temp[i] - base_temp_mean[prefix]

        # ±1도 이내면 냉난방 비가동 (단, 냉방 최소 절전 모드는 항상 유지)
        if temp_diff > 1:
            # 냉방기 작동: 항상 최소 0.2 이상 가동
            cooler_out = max(0.2, temp_diff * 0.4 + door_open[i]*0.4 + duration_hours*0.3)
            cooler_out = np.clip(cooler_out, 0.0, 1.0) / power_efficiency
            temp[i] -= cooler_out * 4.0  # 냉방 효과는 중간 정도
            cooler_output_ratio[i] = cooler_out
            cooler_power_kWh[i] = cooler_out * power_factor * 2.0
        elif temp_diff < -1:
            # 히터는 최대한 사용하지 않도록 절반 이하 제한
            heater_out = (-temp_diff * 0.2 + door_open[i]*0.4 + duration_hours*0.3)
            heater_out = np.clip(heater_out, 0.0, 0.3) / power_efficiency
            temp[i] += heater_out * 1.5
            heater_output_ratio[i] = heater_out
            heater_power_kWh[i] = heater_out * power_factor * 0.5
        else:
            # ±1도 이내일 때: 냉방기 절전 모드 유지
            cooler_out = 0.2 / power_efficiency
            cooler_output_ratio[i] = cooler_out
            cooler_power_kWh[i] = cooler_out * power_factor * 2.0
            temp[i] -= cooler_out * 2.0  # 절전 냉방 효과 낮음

        # 에너지 사용량 및 탄소 배출량 계산
        total_power_kWh[i] = cooler_power_kWh[i] + heater_power_kWh[i]
        carbon_emission_kg[i] = total_power_kWh[i] * 0.5

        # 습도 및 CO2 조정
        humidity[i] = np.clip(humidity[i] - cooler_output_ratio[i]*4.5, 20, 100)
        co2[i] += stock[i]*20 - door_open[i]*30
        co2[i] = max(co2[i], outdoor_co2[i])

    # 데이터프레임 반환
    return pd.DataFrame({
        'datetime': date_range,
        'zone': zone,
        'temperature': temp,
        'humidity': humidity,
        'co2': co2,
        'door_open': door_open,
        'open_duration_minutes': open_duration_minutes,
        'stock_level': stock,
        'cooler_output_ratio': cooler_output_ratio,
        'heater_output_ratio': heater_output_ratio,
        'cooler_power_kWh': cooler_power_kWh,
        'heater_power_kWh': heater_power_kWh,
        'total_power_kWh': total_power_kWh,
        'carbon_emission_kg': carbon_emission_kg,
        'outdoor_temperature': outdoor_temp,
        'outdoor_humidity': outdoor_hum,
        'outdoor_co2': outdoor_co2,
        'storage_size': size,
        'insulation_grade': insulation_levels[prefix],
        'power_efficiency': power_efficiency
    })

# 결과 생성 및 엑셀 저장 (온도: 노란색, 냉난방률: 파란색 강조)
for prefix in ['A', 'B', 'C', 'D']:
    df_list = [generate_zone_data(f"{prefix}-{i}") for i in range(1, 4)]
    df_zone = pd.concat(df_list, ignore_index=True)
    file_path = os.path.join(save_dir, f"냉동창고_시뮬_{prefix}_최종수정.xlsx")

    def highlight_cells(val, col):
        if col in ['temperature', 'outdoor_temperature']:
            return 'background-color: yellow'
        elif col in ['cooler_output_ratio', 'heater_output_ratio']:
            return 'background-color: lightblue'
        return ''

    styled = df_zone.style.apply(lambda row: [
        highlight_cells(val, col) for col, val in row.items()
    ], axis=1)

    styled.to_excel(file_path, index=False, engine='openpyxl')
    print(f"{file_path} 저장 완료")


C:\Users\82108\AppData\Local\Temp\ipykernel_17600\2395951742.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')


C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_A_최종수정.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_B_최종수정.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_C_최종수정.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_D_최종수정.xlsx 저장 완료


In [24]:
import pandas as pd
import numpy as np
import os

# 저장 경로 설정
save_dir = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시"
os.makedirs(save_dir, exist_ok=True)

# 시간 생성 (30일치, 1시간 간격)
date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')

# 구역 이름 생성: A-1 ~ D-3
zones = [f"{z}-{i}" for z in ['A','B','C','D'] for i in range(1,4)]

# 구역별 설정
base_temp_mean = {'A':10, 'B':2, 'C':-18, 'D':5}  # 목표 온도
storage_sizes = {'1':'소형', '2':'중형', '3':'대형'}
insulation_levels = {'A':'상', 'B':'중', 'C':'하', 'D':'중'}
storage_power_factor = {'소형':1.0, '중형':1.5, '대형':2.0}

# 시뮬레이션 함수 정의
def generate_zone_data(zone):
    prefix, idx = zone.split('-')
    rng = np.random.default_rng(seed=int(idx) + ord(prefix))

    # 외부 환경 데이터
    weeks = len(date_range) // (24*7)
    seq_temp = np.concatenate([
        np.linspace(5, 15, 24*7),
        np.linspace(22, 30, 24*7),
        np.linspace(10, 18, 24*7),
        np.linspace(-5, 5, 24*7)
    ])
    outdoor_temp = np.tile(seq_temp, weeks+1)[:len(date_range)]

    seq_hum = np.concatenate([
        np.linspace(40, 60, 24*7),
        np.linspace(70, 90, 24*7),
        np.linspace(40, 60, 24*7),
        np.linspace(30, 50, 24*7)
    ])
    outdoor_hum = np.tile(seq_hum, weeks+1)[:len(date_range)]
    outdoor_co2 = rng.normal(400, 10, len(date_range))

    # 내부 상태 초기화
    temp = np.full(len(date_range), base_temp_mean[prefix], dtype=float)
    humidity = np.full(len(date_range), 60.0, dtype=float)
    stock = np.clip(np.cumsum(rng.normal(0, 0.01, len(date_range))) + 0.5, 0, 1)
    co2 = 400 + stock * 50 + rng.normal(0, 25, len(date_range))
    door_open = rng.binomial(1, 0.5, len(date_range))
    open_duration_minutes = door_open * rng.uniform(5, 60, len(date_range))
    power_efficiency = rng.uniform(0.7, 1.2)

    # 출력 변수 초기화
    cooler_output_ratio = np.zeros(len(date_range))
    heater_output_ratio = np.zeros(len(date_range))
    cooler_power_kWh = np.zeros(len(date_range))
    heater_power_kWh = np.zeros(len(date_range))
    total_power_kWh = np.zeros(len(date_range))
    carbon_emission_kg = np.zeros(len(date_range))

    # 제어 관련 상수
    size = storage_sizes[idx]
    power_factor = storage_power_factor[size]
    ins_factor = {'상': 0.2, '중': 0.5, '하': 1.0}[insulation_levels[prefix]]
    Kp = 0.6

    for i in range(len(date_range)):
        prev_temp = temp[i-1] if i > 0 else base_temp_mean[prefix]
        prev_hum = humidity[i-1] if i > 0 else 60.0
        duration_hours = open_duration_minutes[i] / 60

        # 문 열림에 따른 열유입
        if door_open[i]:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.6 * duration_hours
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.5 * duration_hours
        else:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.2
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.05

        # 🔥 내부 자연 발열 효과 추가
        internal_heat_gain = power_factor * 0.15
        temp[i] += internal_heat_gain

        # 온도 제어: ±1도 범위 유지
        temp_diff = temp[i] - base_temp_mean[prefix]

        # 냉방기 제어
        if temp_diff > 1.0:
            cooler_out = max(0.2, Kp * temp_diff)
            temp[i] -= cooler_out * 4.0
        else:
            cooler_out = 0.2  # 항상 절전 모드 유지

        # 히터 제어 (A, D구역만)
        if prefix in ['A', 'D'] and temp_diff < -1.0:
            heater_out = min(0.8, -Kp * temp_diff)
            temp[i] += heater_out * 2.5
        else:
            heater_out = 0.0

        # 에너지 및 배출량 계산
        cooler_output_ratio[i] = np.clip(cooler_out / power_efficiency, 0.0, 2.0)
        heater_output_ratio[i] = heater_out
        cooler_power_kWh[i] = cooler_output_ratio[i] * power_factor * 2.0
        heater_power_kWh[i] = heater_output_ratio[i] * power_factor * 1.5
        total_power_kWh[i] = cooler_power_kWh[i] + heater_power_kWh[i]
        carbon_emission_kg[i] = total_power_kWh[i] * 0.5

        # 습도 및 CO2 조정
        humidity[i] = np.clip(humidity[i] - cooler_output_ratio[i]*4.5, 20, 100)
        co2[i] += stock[i]*20 - door_open[i]*30
        co2[i] = max(co2[i], outdoor_co2[i])

    return pd.DataFrame({
        'datetime': date_range,
        'zone': zone,
        'temperature': temp,
        'humidity': humidity,
        'co2': co2,
        'door_open': door_open,
        'open_duration_minutes': open_duration_minutes,
        'stock_level': stock,
        'cooler_output_ratio': cooler_output_ratio,
        'heater_output_ratio': heater_output_ratio,
        'cooler_power_kWh': cooler_power_kWh,
        'heater_power_kWh': heater_power_kWh,
        'total_power_kWh': total_power_kWh,
        'carbon_emission_kg': carbon_emission_kg,
        'outdoor_temperature': outdoor_temp,
        'outdoor_humidity': outdoor_hum,
        'outdoor_co2': outdoor_co2,
        'storage_size': size,
        'insulation_grade': insulation_levels[prefix],
        'power_efficiency': power_efficiency
    })

# 전체 구역별로 실행 및 저장
for prefix in ['A', 'B', 'C', 'D']:
    df_list = [generate_zone_data(f"{prefix}-{i}") for i in range(1, 4)]
    df_zone = pd.concat(df_list, ignore_index=True)
    file_path = os.path.join(save_dir, f"냉동창고_시뮬_{prefix}_히터_자연발열_±1도제어.xlsx")
    df_zone.to_excel(file_path, index=False)
    print(f"{file_path} 저장 완료")


C:\Users\82108\AppData\Local\Temp\ipykernel_17600\86754167.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')


C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_A_히터_자연발열_±1도제어.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_B_히터_자연발열_±1도제어.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_C_히터_자연발열_±1도제어.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_D_히터_자연발열_±1도제어.xlsx 저장 완료


| 데이터 시점 (시간 기준) | 실제 시점 (4시간 = 하루 기준) | 효과            |
| -------------- | ------------------- | ------------- |
| 0\~3           | 실제 1일 오전            | 겨울, 새벽 온도     |
| 4\~7           | 실제 1일 낮             | 겨울, 낮 온도      |
| ...            | ...                 | ...           |
| 336\~671       | 실제 14일              | → 15일째부터 봄 시작 |


In [9]:
import pandas as pd
import numpy as np
import os

# 저장 경로 설정
save_dir = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시"
os.makedirs(save_dir, exist_ok=True)

# 1시간 간격 데이터 생성 (60일 = 1440시간)
date_range = pd.date_range(start="2025-01-01", periods=24 * 60, freq='H')

# 계절 구분 함수 (4시간 = 1일 간주 → 336시간마다 계절 변경)
def get_season_by_index(index):
    period = index // 336
    seasons = ['winter', 'spring', 'summer', 'autumn']
    return seasons[period % 4]

# 외기 환경 생성 함수
def generate_outdoor_conditions(date_range):
    outdoor_temp = []
    outdoor_hum = []
    rainfall_mm = []

    for i, dt in enumerate(date_range):
        hour_in_day = i % 6  # 4시간 단위 하루 기준으로 위치
        season = get_season_by_index(i)

        # 계절별 온도 설정
        if season in ['spring', 'autumn']:
            base_temp = 12 if season == 'spring' else 15
            diurnal_range = 7
        elif season == 'summer':
            base_temp = 28
            diurnal_range = 15
        else:  # winter
            base_temp = 2
            diurnal_range = 15

        # 6개 주기로 일교차 반영 (4시간 = 하루)
        theta = 2 * np.pi * (hour_in_day / 6)
        temp_variation = diurnal_range * np.sin(theta - np.pi/2)
        temp = base_temp + temp_variation

        # 강수량(mm) 및 그에 따른 조정
        rain_mm = np.random.exponential(1.5) if np.random.rand() < 0.2 else 0
        if rain_mm > 0:
            temp -= 3

        humidity = 60 + rain_mm * 5 + np.random.normal(0, 5)

        outdoor_temp.append(temp)
        outdoor_hum.append(np.clip(humidity, 30, 100))
        rainfall_mm.append(rain_mm)

    return np.array(outdoor_temp), np.array(outdoor_hum), np.array(rainfall_mm)

# 외부 데이터 생성
outdoor_temp_global, outdoor_hum_global, rainfall_global = generate_outdoor_conditions(date_range)

# 기본 설정값
base_temp_mean = {'A':10, 'B':2, 'C':-18, 'D':5}
storage_sizes = {'1':'소형', '2':'중형', '3':'대형'}
insulation_levels = {'A':'상', 'B':'중', 'C':'하', 'D':'중'}
storage_power_factor = {'소형':1.0, '중형':1.5, '대형':2.0}

# 시뮬레이션 함수
def generate_zone_data(zone):
    prefix, idx = zone.split('-')
    rng = np.random.default_rng(seed=int(idx) + ord(prefix))

    temp = np.full(len(date_range), base_temp_mean[prefix], dtype=float)
    humidity = np.full(len(date_range), 60.0, dtype=float)
    stock = np.clip(np.cumsum(rng.normal(0, 0.01, len(date_range))) + 0.5, 0, 1)
    co2 = 400 + stock * 50 + rng.normal(0, 25, len(date_range))
    door_open = rng.binomial(1, 0.5, len(date_range))
    open_duration_minutes = door_open * rng.uniform(5, 60, len(date_range))
    power_efficiency = rng.uniform(0.7, 1.2)

    cooler_output_ratio = np.zeros(len(date_range))
    heater_output_ratio = np.zeros(len(date_range))
    cooler_power_kWh = np.zeros(len(date_range))
    heater_power_kWh = np.zeros(len(date_range))
    total_power_kWh = np.zeros(len(date_range))
    carbon_emission_kg = np.zeros(len(date_range))

    size = storage_sizes[idx]
    power_factor = storage_power_factor[size]
    ins_factor = {'상': 0.2, '중': 0.5, '하': 1.0}[insulation_levels[prefix]]
    Kp = 0.6

    for i in range(len(date_range)):
        prev_temp = temp[i-1] if i > 0 else base_temp_mean[prefix]
        prev_hum = humidity[i-1] if i > 0 else 60.0
        duration_hours = open_duration_minutes[i] / 60

        out_temp = outdoor_temp_global[i]
        out_hum = outdoor_hum_global[i]
        out_co2 = 400

        if door_open[i]:
            temp[i] = prev_temp + (out_temp - prev_temp) * ins_factor * 0.6 * duration_hours
            humidity[i] = prev_hum + (out_hum - prev_hum) * 0.5 * duration_hours
        else:
            temp[i] = prev_temp + (out_temp - prev_temp) * ins_factor * 0.2
            humidity[i] = prev_hum + (out_hum - prev_hum) * 0.05

        temp[i] += power_factor * 0.15

        temp_diff = temp[i] - base_temp_mean[prefix]
        if temp_diff > 1.0:
            cooler_out = max(0.2, Kp * temp_diff)
            temp[i] -= cooler_out * 4.0
        else:
            cooler_out = 0.2

        if prefix in ['A', 'D'] and temp_diff < -1.0:
            heater_out = min(0.8, -Kp * temp_diff)
            temp[i] += heater_out * 2.5
        else:
            heater_out = 0.0

        cooler_output_ratio[i] = np.clip(cooler_out / power_efficiency, 0.0, 2.0)
        heater_output_ratio[i] = heater_out
        cooler_power_kWh[i] = cooler_output_ratio[i] * power_factor * 2.0
        heater_power_kWh[i] = heater_output_ratio[i] * power_factor * 1.5
        total_power_kWh[i] = cooler_power_kWh[i] + heater_power_kWh[i]
        carbon_emission_kg[i] = total_power_kWh[i] * 0.5

        humidity[i] = np.clip(humidity[i] - cooler_output_ratio[i]*4.5, 20, 100)
        co2[i] += stock[i]*20 - door_open[i]*30
        co2[i] = max(co2[i], out_co2)

    return pd.DataFrame({
        'datetime': date_range,
        'zone': zone,
        'temperature': temp,
        'humidity': humidity,
        'co2': co2,
        'door_open': door_open,
        'open_duration_minutes': open_duration_minutes,
        'stock_level': stock,
        'cooler_output_ratio': cooler_output_ratio,
        'heater_output_ratio': heater_output_ratio,
        'cooler_power_kWh': cooler_power_kWh,
        'heater_power_kWh': heater_power_kWh,
        'total_power_kWh': total_power_kWh,
        'carbon_emission_kg': carbon_emission_kg,
        'outdoor_temperature': outdoor_temp_global,
        'outdoor_humidity': outdoor_hum_global,
        'rainfall_mm': rainfall_global,
        'storage_size': size,
        'insulation_grade': insulation_levels[prefix],
        'power_efficiency': power_efficiency
    })

# 구역별 저장
for prefix in ['A', 'B', 'C', 'D']:
    df_list = [generate_zone_data(f"{prefix}-{i}") for i in range(1, 4)]
    df_zone = pd.concat(df_list, ignore_index=True)
    file_path = os.path.join(save_dir, f"냉동창고_시뮬_{prefix}_60일_4시간하루기준.xlsx")
    df_zone.to_excel(file_path, index=False)
    print(f"{file_path} 저장 완료")


C:\Users\82108\AppData\Local\Temp\ipykernel_17468\2075160568.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-01-01", periods=24 * 60, freq='H')


C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_A_60일_4시간하루기준.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_B_60일_4시간하루기준.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_C_60일_4시간하루기준.xlsx 저장 완료
C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_D_60일_4시간하루기준.xlsx 저장 완료


In [13]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# 🔹 엑셀 파일 경로 (수정 필요)
file_path = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_D_60일_4시간하루기준.xlsx"

# 🔹 색상 정의 (RGB 형식)
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # 노란색
blue_fill = PatternFill(start_color="00B0F0", end_color="00B0F0", fill_type="solid")    # 파란색

# 🔹 색칠할 컬럼명
yellow_columns = ['temperature', 'outdoor_temperature']
blue_columns = ['cooler_output_ratio', 'heater_output_ratio']

# 🔹 파일 열기 및 시트 접근
wb = load_workbook(file_path)
ws = wb.active  # 첫 번째 시트 기준

# 🔹 컬럼 인덱스 찾기
header = [cell.value for cell in ws[1]]

for col_name in yellow_columns + blue_columns:
    if col_name not in header:
        print(f"❌ 컬럼 '{col_name}'을 찾을 수 없습니다.")
        continue

    col_idx = header.index(col_name) + 1  # 엑셀은 1부터 시작

    # 셀 색칠하기 (2행부터 끝까지)
    fill = yellow_fill if col_name in yellow_columns else blue_fill
    for row in range(2, ws.max_row + 1):
        ws.cell(row=row, column=col_idx).fill = fill

# 🔹 저장
wb.save(file_path)
print("✅ 열 색칠 완료 및 저장됨:", file_path)


✅ 열 색칠 완료 및 저장됨: C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시\냉동창고_시뮬_D_60일_4시간하루기준.xlsx


## trnsys 18 데이터 만들기

In [ ]:
import pandas as pd
import numpy as np
import os

# 저장 경로 설정
save_dir = r"C:\Users\82108\Desktop\LX 판토스 공장 시뮬 예시"
os.makedirs(save_dir, exist_ok=True)

# --- 여기부터 generate_zone_data 함수 원본 전체 복사 붙여넣기 ---
def generate_zone_data(zone):
    prefix, idx = zone.split('-')
    rng = np.random.default_rng(seed=int(idx) + ord(prefix))

    date_range = pd.date_range(start="2025-01-01", periods=24 * 30, freq='H')

    weeks = len(date_range) // (24*7)
    seq_temp = np.concatenate([
        np.linspace(5, 15, 24*7),
        np.linspace(22, 30, 24*7),
        np.linspace(10, 18, 24*7),
        np.linspace(-5, 5, 24*7)
    ])
    outdoor_temp = np.tile(seq_temp, weeks+1)[:len(date_range)]

    seq_hum = np.concatenate([
        np.linspace(40, 60, 24*7),
        np.linspace(70, 90, 24*7),
        np.linspace(40, 60, 24*7),
        np.linspace(30, 50, 24*7)
    ])
    outdoor_hum = np.tile(seq_hum, weeks+1)[:len(date_range)]
    outdoor_co2 = rng.normal(400, 10, len(date_range))

    temp = np.full(len(date_range), {'A':10, 'B':2, 'C':-18, 'D':5}[prefix], dtype=float)
    humidity = np.full(len(date_range), 60.0, dtype=float)
    stock = np.clip(np.cumsum(rng.normal(0,0.01,len(date_range)))+0.5, 0,1)
    co2 = 400 + stock*50 + rng.normal(0,25,len(date_range))
    door_open = rng.binomial(1, 0.5, len(date_range))
    open_duration_minutes = door_open * rng.uniform(5,60,len(date_range))
    power_efficiency = rng.uniform(0.7, 1.2)

    cooler_output_ratio = np.zeros(len(date_range))
    heater_output_ratio = np.zeros(len(date_range))
    cooler_power_kWh = np.zeros(len(date_range))
    heater_power_kWh = np.zeros(len(date_range))
    total_power_kWh = np.zeros(len(date_range))
    carbon_emission_kg = np.zeros(len(date_range))

    storage_sizes = {'1':'소형', '2':'중형', '3':'대형'}
    storage_power_factor = {'소형':1.0, '중형':1.5, '대형':2.0}
    insulation_levels = {'A':'상', 'B':'중', 'C':'하', 'D':'중'}

    size = storage_sizes[idx]
    power_factor = storage_power_factor[size]
    ins_factor = {'상': 0.2, '중': 0.5, '하': 1.0}[insulation_levels[prefix]]

    for i in range(len(date_range)):
        prev_temp = temp[i-1] if i > 0 else {'A':10, 'B':2, 'C':-18, 'D':5}[prefix]
        prev_hum = humidity[i-1] if i > 0 else 60.0
        duration_hours = open_duration_minutes[i] / 60

        if door_open[i]:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.6 * duration_hours
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.5 * duration_hours
        else:
            temp[i] = prev_temp + (outdoor_temp[i] - prev_temp) * ins_factor * 0.05
            humidity[i] = prev_hum + (outdoor_hum[i] - prev_hum) * 0.05

        temp_diff = temp[i] - {'A':10, 'B':2, 'C':-18, 'D':5}[prefix]

        if temp_diff > 0:
            cooler_out = ((temp_diff * 0.3) + (door_open[i] * 0.4) + (duration_hours * 0.3))
            cooler_out = np.clip(cooler_out, 0.0, 1.0) / power_efficiency
            temp[i] -= cooler_out * 2.5
            cooler_output_ratio[i] = cooler_out
            cooler_power_kWh[i] = cooler_out * power_factor
        elif temp_diff < 0:
            heater_out = ((-temp_diff * 0.4) + (door_open[i] * 0.4) + (duration_hours * 0.3))
            heater_out = np.clip(heater_out, 0.0, 1.0) / power_efficiency
            temp[i] += heater_out * 2.0
            heater_output_ratio[i] = heater_out
            heater_power_kWh[i] = heater_out * power_factor
        else:
            cooler_out = 0.0
            heater_out = 0.0

        total_power_kWh[i] = cooler_power_kWh[i] + heater_power_kWh[i]
        carbon_emission_kg[i] = total_power_kWh[i] * 0.5

        humidity[i] -= cooler_output_ratio[i] * 4.5
        humidity[i] = np.clip(humidity[i], 20, 100)
        co2[i] += stock[i] * 20 - door_open[i] * 30
        co2[i] = max(co2[i], outdoor_co2[i])

    return pd.DataFrame({
        'zone': zone,
        'datetime': date_range,
        'temperature': temp,
        'humidity': humidity,
        'co2': co2,
        'door_open': door_open,
        'open_duration_minutes': open_duration_minutes,
        'stock_level': stock,
        'cooler_output_ratio': cooler_output_ratio,
        'heater_output_ratio': heater_output_ratio,
        'cooler_power_kWh': cooler_power_kWh,
        'heater_power_kWh': heater_power_kWh,
        'total_power_kWh': total_power_kWh,
        'carbon_emission_kg': carbon_emission_kg,
        'outdoor_temperature': outdoor_temp,
        'outdoor_humidity': outdoor_hum,
        'outdoor_co2': outdoor_co2,
        'storage_size': size,
        'insulation_grade': insulation_levels[prefix],
        'power_efficiency': power_efficiency
    })

# TRNSYS Type99용 파일 생성 함수
def save_trnsys_type99_file(df, file_path):
    n = len(df)
    time_col = np.arange(1, n+1)
    data = pd.DataFrame({
        'Time': time_col,
        'Outdoor_Temp': df['outdoor_temperature'].values,
        'Outdoor_Humidity': df['outdoor_humidity'].values,
        'Direct_Solar': np.zeros(n),
        'Diffuse_Solar': np.zeros(n),
        'Wind_Speed': np.zeros(n),
        'Wind_Direction': np.zeros(n)
    })
    data.to_csv(file_path, sep=' ', index=False, header=False, float_format='%.2f')

# 예시: A, B, C, D 구역 데이터 저장
for prefix in ['A', 'B', 'C', 'D']:
    df_list = [generate_zone_data(f"{prefix}-{i}") for i in range(1, 4)]
    df_zone = pd.concat(df_list, ignore_index=True)
    txt_file_path = os.path.join(save_dir, f"냉동창고_TRNSYS_Type99_{prefix}.txt")
    save_trnsys_type99_file(df_zone, txt_file_path)
    print(f"{txt_file_path} 저장 완료")


C:\Users\82108\AppData\Local\Temp\ipykernel_17600\1283493449.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-01-01", periods=24*30, freq='H')


C:\Users\82108\Desktop\LX_판토스_시뮬_데이터\냉동창고_시뮬_A.txt 저장 완료
C:\Users\82108\Desktop\LX_판토스_시뮬_데이터\냉동창고_시뮬_B.txt 저장 완료
C:\Users\82108\Desktop\LX_판토스_시뮬_데이터\냉동창고_시뮬_C.txt 저장 완료
C:\Users\82108\Desktop\LX_판토스_시뮬_데이터\냉동창고_시뮬_D.txt 저장 완료


In [5]:
file_path = os.path.join(save_dir, f"냉동창고_시뮬_{prefix}.txt")
print("파일 저장 경로:", file_path)


파일 저장 경로: C:\Users\82108\Desktop\LX_판토스_시뮬_데이터\냉동창고_시뮬_D.txt
